# Markov-Chain Monte Carlo Simulation
The purpose of this notebook is to outline how to perform a Markov-Chain Monte Carlo (MCMC) simulation using the `potions` library. MCMC is a powerful technique used in statistics and machine learning for sampling from complex probability distributions. It's particularly useful in this case because it provides us a way to explore the parameter space of our model without having to evaluate every possible combination, giving us both the optimal parameter sets and also the uncertainty around the parameters.

## Import libraries and load data

In [1]:
import numpy as np
import pandas as pd 
from pandas import DataFrame, Series
import emcee
import potions as pt
import warnings
from multiprocessing import Pool
warnings.filterwarnings("ignore")

In [2]:
file_path: str = "../input/Sleepers_Results.txt"
input_df = pd.read_csv(file_path, sep="\\s+", index_col="Date", parse_dates=True)
forc = pt.ForcingData(
    precip=input_df["Precipitation"],
    temp=input_df["Temperature"],
    pet=input_df["PET"]
)
meas_streamflow = input_df["Qobs"]

ModelType: type[pt.Model] = pt.HbvModel
include_lapse_rates: bool = False

## Set up the Markov-Chain Monte Carlo (MCMC) simulation
In an MCMC simulation we need to define a model, which is essentially a function that takes in parameters and returns the likelihood of getting the objective function (like NSE, KGE, ...) given those parameters. We also need to specify the prior distribution over the parameters, which represents our beliefs about what values are likely for each parameter before we start the simulation. To keep things simple, we can just use a uniform prior distribution for all parameters.

For an MCMC simulation, there are three functions that we need to define:
- `log_prior`: Ensures that all of the parameters are within the bounds that we set
- `log_likelihood`: Calculates the likelihood of getting the objective function given the current parameter values
- `log_probability`: Combines the log prior and log likelihood to get the total probability of the current parameter values. This function makes sure that both the parameters are acceptable and also how good the model fit is.

In [3]:
def log_prior(params: np.ndarray) -> float:
    for param, (min_val, max_val) in zip(params, ModelType.default_parameter_ranges(include_lapse_rates=include_lapse_rates).values()):
        if param < min_val or param > max_val:
            return -np.inf
    return 0.0

def log_likelihood(theta: np.ndarray) -> float:
    res: dict = ModelType.from_array(theta).run(
        forc=forc,
        meas_streamflow=meas_streamflow
    )
    return np.exp(res["kge"] + res["nse"])

    
def log_probability(theta: np.ndarray) -> tuple[float, tuple[float, float]]:
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf, (np.nan, np.nan)
    else:
        model_res = ModelType.from_array(theta).run(
            forc=forc,
            meas_streamflow=meas_streamflow
        )
        kge: float 
        nse: float 
        kge, nse = model_res["kge"], model_res["nse"] # type: ignore
        log_like_value: float = np.exp(kge + nse)

        return lp + log_like_value, (kge, nse)

## Run the Markov-Chain Monte Carlo Simulation

In [7]:
mcmc_res = pt.HbvLateralModel.mcmc(
    forc=forc,
    meas_streamflow=meas_streamflow,
    num_samples=5000
)

100%|██████████| 5000/5000 [04:45<00:00, 17.49it/s]


In [8]:
mcmc_res[1].max()

kge     0.725699
nse     0.701214
bias    1.012555
dtype: float64

In [4]:
bounds = ModelType.default_parameter_ranges(include_lapse_rates=include_lapse_rates)
initial_guess = np.array([0.5 * (x[0] + x[1]) for x in bounds.values()]) # Have the initial guess be in the middle of the parameter ranges
param_ranges = np.array([x[1] - x[0] for x in bounds.values()])
ndim: int = initial_guess.size 
num_walkers: int = 3 * ndim 
pos = [initial_guess + 0.5 * param_ranges * np.random.randn(ndim) for i in range(num_walkers)]

In [5]:
with Pool() as pool:
    sampler = emcee.EnsembleSampler(
        num_walkers, 
        ndim, 
        log_prob_fn=log_probability,
        pool=pool
    )
    mc_result = sampler.run_mcmc(
        pos, 
        1_000, 
        progress=True
    )

100%|██████████| 1000/1000 [00:30<00:00, 32.42it/s]


In [8]:
type(mc_result)

emcee.state.State

In [6]:
blob_arr: np.ndarray = sampler.get_blobs()
blob_arr = blob_arr.reshape((blob_arr.shape[0] * blob_arr.shape[1], blob_arr.shape[2]))
obj_func_df = DataFrame(blob_arr, columns=["kge", "nse"])

In [7]:
sample_arr: np.ndarray = sampler.get_chain()
sample_arr: np.ndarray = sample_arr.reshape((sample_arr.shape[0] * sample_arr.shape[1], sample_arr.shape[2]))
sample_df = DataFrame(sample_arr, columns=ModelType().parameter_names())

In [58]:
top_obj_df = obj_func_df.loc[obj_func_df.kge >= 0.875]
top_sample_df = sample_df.loc[sample_df.index.isin(top_obj_df.index)]
print(f"Number of samples: {len(top_obj_df)}")

Number of samples: 1459


In [60]:
top_ser: Series = top_obj_df.loc[top_obj_df.kge.idxmax()] # type: ignore
top_params: Series = top_sample_df.loc[top_obj_df.kge.idxmax()] # type: ignore

In [61]:
top_params

snow.tt           0.826773
snow.fmax         3.138050
soil.fc         473.103551
soil.lp           1.542104
soil.beta         0.208132
soil.k0           0.500411
soil.thr         48.673961
shallow.k         0.086153
shallow.perc      3.407920
deep.k            0.099947
Name: 135928, dtype: float64

In [64]:
rel_errs = (top_sample_df - top_params) / (top_params)

In [65]:
rel_errs.sum(axis=1).max()

np.float64(6.040954987897538)

In [63]:
errs.head()

,snow.tt,snow.fmax,soil.fc,soil.lp,soil.beta,soil.k0,soil.thr,shallow.k,shallow.perc,deep.k
15403,-0.117136,-0.141521,-198.926627,0.657519,0.173779,0.077714,15.067347,-0.003878,-1.298237,-0.058715
15433,-0.117136,-0.141521,-198.926627,0.657519,0.173779,0.077714,15.067347,-0.003878,-1.298237,-0.058715
17513,-0.076584,-0.460928,266.321241,-0.166599,-0.023519,0.084377,18.530976,-0.008834,-2.166032,-0.052962
17543,-0.076584,-0.460928,266.321241,-0.166599,-0.023519,0.084377,18.530976,-0.008834,-2.166032,-0.052962
17573,-0.076584,-0.460928,266.321241,-0.166599,-0.023519,0.084377,18.530976,-0.008834,-2.166032,-0.052962


In [39]:
sample_df.head()

,kge,nse
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [ ]:
obj_func_df.kge.describe()

count    280052.000000
mean          0.512304
std           0.225751
min          -0.491455
25%           0.384440
50%           0.556821
75%           0.676073
max           0.909876
Name: kge, dtype: float64

In [ ]:
top_vals = obj_func_df.loc[obj_func_df]

In [24]:
mc_result.coords.shape

(20, 10)

In [26]:
sampler.get_blobs().shape

(1000, 20, 2)

In [22]:
obj_func_df.shape

(20, 2)